<a href="https://colab.research.google.com/github/InbarFabian/AppliedFinal/blob/main/AppliedFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Statistics